# Tuning GNN-ePC-SAFT

## Starting point

In [ ]:
#    import os
#    import torch
#    os.environ['TORCH'] = torch.__version__
#    print(torch.__version__)

#   !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
%pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
%pip install rdkit torchmetrics ml-collections polars clu contextlib2 -q --no-deps
!mkdir pcsaft
!cp -a /kaggle/input/pcsaftpkg/PC-SAFT-1.4.1/. /kaggle/working/pcsaft
%pip install ./pcsaft -q
%pip install ogb -q

!pip show pcsaft | grep -E "Version|Name"

!lscpu | grep -E "Model name|Core|core" 

!nvidia-smi

In [ ]:
%pip install hpbandster ConfigSpace -q
%pip install ray -U -q

In [ ]:
import os, shutil

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("ghtk")
secret_value_1 = user_secrets.get_secret("kaggle")
secret_value_2 = user_secrets.get_secret("username")
secret_value_3 = user_secrets.get_secret("wandbkey")
os.environ["KAGGLE_USERNAME"]=secret_value_2
os.environ["KAGGLE_KEY"]=secret_value_1
import wandb
os.environ['WANDB_API_KEY'] = secret_value_3
wandb.login()

In [ ]:
# for the cloud
!git clone https://{secret_value_0}@github.com/wildsonbbl/GNN-ePC-SAFT.git --branch san --single-branch -q
%cd GNN-ePC-SAFT/

# previous running results
!cp -a /kaggle/input/gnnepcsaft/tuning/root/* ~

## Tuning

In [ ]:
!python -m train.tuner --workdir=/kaggle/working/GNN-ePC-SAFT --config=configs/default.py \
--dataset=ramirez --tags="batch 2" \
--num_cpu=4 --num_gpus=1 --num_init_gpus=1 \
--num_samples=50 --time_budget_s=39600 --verbose=0\
--config.num_train_steps=11000 --config.log_every_steps=1000 --config.model=PNA2 \
--get_result=False --restoredir=/root/ray_results/gnnpcsaft/

### Clear workspace

In [ ]:
%cd ../
!rm -rf *

### Zip results

In [ ]:
!rm -rf ~/ray*/gnn*/tune*/check* ~/ray*/gnn*/tune*/wandb
!zip -r tuning.zip ~/ray* -q